In [2]:
import itertools
import math
import pickle
import time

from simulation import simulate
from algs import *
from payoff import *
from plotting import *
from util import *

In [3]:
steps = 2000
narms = 2
nsims = 4

strategies = [ucb1]
strategies_alt = [thompson_sampling]
nagents = [2000, 500, 100, 50]
lifespans = [5, 20, 50, 200]

# Negative externalities

$$ Q(\theta, f) = \frac{\theta}{1+Lf}$$

In [4]:
def neg_ext(Ls, strategies, strategies_alt, lifespans, nagents, mixes, steps=2000):
    t_start = time.time()
    
    size_cross = len(Ls) * len(strategies) * len(lifespans) * len(nagents) * len(strategies_alt) * len(mixes)
    print("Starting {} simulations".format(size_cross))

    for itr, (L, strategy, strategy_alt ,lifespan, nagent, mix) in enumerate(itertools.product(Ls, strategies, strategies_alt, lifespans, nagents, mixes)):
        config = {"k": narms, 
                  "types": [(3, 1), (1, 3)],
                  "nagents": nagent, 
                  "beta": lifespan_to_beta(lifespan), 
                  "strategy": strategy, 
                  "strategy_alt": strategy_alt,
                  "payoff": create_neg_externatilities(L),
                  "mix": mix}

        fname = "neg_ext_L{}_S{}_S{}_A{}_K{}_LS{}_M{}".format(int(10*L),
                                                      config["strategy"].__name__, 
                                                      config["strategy_alt"].__name__, 
                                                      config["nagents"], 
                                                      config["k"], 
                                                      lifespan,
                                                      config["mix"])

        outcome = [simulate(config, steps) for _ in range(nsims)]
        dump(outcome, fname)

        if itr % max(40, int(size_cross / 10)+1):
            progress(itr, size_cross, t_start)

    print("Total time: {:.2f} seconds".format(time.time() - t_start))

In [4]:
# Figure 1
neg_ext([0.2, 3, 7, 20], [ucb3], [ucb3] ,[50], [2000], [0], steps)

Starting 4 simulations
	  50.0% 	 172.92s 	 172.92s
	  75.0% 	 258.33s 	 86.11s
	 100.0% 	 344.07s 	  0.00s
Total time: 344.07 seconds


In [6]:
# Figure 2
neg_ext([20], [ucb1], [ucb1], [200], [50, 100, 500, 2000], [0], steps)

Starting 4 simulations
	  50.0% 	 12.50s 	 12.50s
	  75.0% 	 55.39s 	 18.46s
	 100.0% 	 220.74s 	  0.00s
Total time: 220.74 seconds


# Positive externalities

In [7]:
def pos_ext(strategies, strategies_alt ,nagents, lifespans, mixes, steps=2000):
    size_cross = len(strategies) * len(nagents) * len(lifespans) * len(strategies_alt) * len(mixes)
    t_start = time.time()
    print("Starting {} simulations".format(size_cross))

    for itr, (strategy, strategy_alt, nagent, lifespan, mix) in enumerate(itertools.product(strategies, strategies_alt, nagents, lifespans, mixes)):
        config = {"k": narms, 
                  "types": [(1, 1), (1, 1)],
                  "nagents": nagent, 
                  "beta": lifespan_to_beta(lifespan), 
                  "strategy": strategy, 
                  "strategy_alt": strategy_alt, 
                  "payoff": pos_externalities,
                  "mix": mix}

        fname = "pos_ext_S{}_S{}_A{}_K{}_LS{}_M{}".format(config["strategy"].__name__,
                                                      config["strategy_alt"].__name__,
                                                    config["nagents"], 
                                                    config["k"], 
                                                    lifespan,
                                                    config["mix"])

        outcome = [simulate(config, steps) for _ in range(nsims)]
        dump(outcome, fname)

        if itr % max(40, int(size_cross / 10)+1):
            progress(itr, size_cross, t_start)

    print("Total time: {:.2f} seconds".format(time.time() - t_start))

In [18]:
# Figure 3 and 4
pos_ext([ucb1], [ucb1], nagents, [10, 200], [0], steps)

Starting 8 simulations
	  25.0% 	 328.82s 	 986.45s
	  37.5% 	 367.61s 	 612.68s
	  50.0% 	 409.73s 	 409.73s
	  62.5% 	 417.96s 	 250.78s
	  75.0% 	 426.29s 	 142.10s
	  87.5% 	 430.52s 	 61.50s
	 100.0% 	 435.09s 	  0.00s
Total time: 435.09 seconds


# Coordination game

In [15]:
def coord_game(gammas, strategies, strategies_alt ,nagents, lifespans, mixes):
    size_cross = len(gammas) * len(strategies) * len(nagents) * len(lifespans) * len(strategies_alt) * len(mixes)
    t_start = time.time()
    print("Starting {} simulations".format(size_cross))

    for itr, (gamma, strategy, strategy_alt, nagent, lifespan, mix) in enumerate(itertools.product(gammas, strategies, strategies_alt, nagents, lifespans, mixes)):
        config = {"k": narms, 
                  "types": [(1, 1), (1, 1)],
                  "nagents": nagent, 
                  "beta": lifespan_to_beta(lifespan), 
                  "strategy": strategy, 
                  "strategy_alt": strategy_alt,
                  "payoff": create_coord_game(gamma),
                  "mix": mix}

        fname = "coordgame_G{}_S{}_S{}_A{}_K{}_LS{}_M{}".format(int(gamma*100),
                                                      config["strategy"].__name__, 
                                                      config["strategy_alt"].__name__,
                                                      config["nagents"], 
                                                      config["k"], 
                                                      lifespan,
                                                      config["mix"])

        outcome = [simulate(config, steps) for _ in range(nsims)]
        dump(outcome, fname)

        if itr % max(40, int(size_cross / 10)+1):
            progress(itr, size_cross, t_start)

    print("Total time: {:.2f} seconds".format(time.time() - t_start))

In [17]:
# Figure 5
coord_game([0.5], [ucb2], [ucb2] ,[2000], lifespans, [0])

Starting 4 simulations
	  50.0% 	 283.78s 	 283.78s
	  75.0% 	 424.77s 	 141.59s
	 100.0% 	 562.28s 	  0.00s
Total time: 562.28 seconds
